Done:

- Генератор 1000 строк
- Юниформ распределение алфавита
- Длина слова 10-100-1000-10000
- Длина паттерна 5
- Посчитать среднее/макс/мин на каждой длине слова
- Naive

Undone:

- RK
- BMH
- Отчет(оформление)

In [1]:
#imports:
import numpy as np
import pandas as pd

#constants
PATTERN_LENGTH = 5

In [2]:
def string_generator(alp_length, str_length):
    string = []
    for i in range (str_length):
        string.append(chr(97 + np.random.randint(0,alp_length)))
    #print(string)
    return string

def testset_generator(alp_length, str_length):
    testset = []
    for i in range (1000):
        testset.append(string_generator(alp_length, str_length))
    return testset

def iterations_computation(algorithm, pattern, testset):
    max_count = avg_count = sum_count =  0
    min_count = 10e5
    frame = pd.DataFrame({'str': testset})
    for i in range (len(testset)):
        string = frame.iloc[i]
        result = string.apply(lambda x: algorithm(string[0], len(string[0]), pattern))
        count = result[0][1]
        sum_count = sum_count + count
        if count >= max_count:
            max_count = count
        if count <= min_count:
            min_count = count
    avg_count = sum_count / len(testset)
    return max_count, min_count, avg_count
            

In [ ]:
set10 = testset_generator(4,10)
set100 = testset_generator(4,100)
set1000 = testset_generator(4,1000)
set10000 = testset_generator(4,10000)


In [ ]:
def Naive(string, str_length, pattern):
    shifts_list = []
    shifts_count = 0
    comparisons_count = 0
    for i in range(str_length - PATTERN_LENGTH + 1):#нужен ли +1???
        shifts_count = shifts_count + 1
        flag = True
        for j in range (PATTERN_LENGTH):
            comparisons_count = comparisons_count + 1
            if i + j < str_length and string[i + j] != pattern[j]:
                flag = False
                break
        if flag:
            shifts_list.append(i)
    return shifts_list, shifts_count+comparisons_count 

In [ ]:
#a = ['a','c','a','b','b','c','c','c','c','b']
#['c','c','c','c']
pattern = string_generator(4,5)

In [ ]:
max10, min10, avg10 = iterations_computation(Naive, pattern, set10)
print(max10, min10, avg10)
max100, min100, avg100 =iterations_computation(Naive, pattern, set100)
print(max100, min100, avg100)
max1000, min1000, avg1000 =iterations_computation(Naive, pattern, set1000)
print(max1000, min1000, avg1000)
max10000, min10000, avg10000 =iterations_computation(Naive, pattern, set10000)
print(max10000, min10000, avg10000)